# TextRank 알고리즘 적용 위한 스텝
1. 텍스트 수집 (완)
2. 전처리
  - 불용어 처리 (re 사용)
3. tf-idf 통해 문서 내 특정 단어의 중요도 분석
  - 텍스트를 문장 단위로 분리
  - koNLPY twitter 활용해 형태소 태깅 + 조사 및 어간 분리 통해 명사 반환
  - TfidfVectorizer()와 CountVectorizer() 활용하여 tf, df 계산
    - 이때 word2vec, doc2vec 등을 결합하여 문서 간 유사도 계산 가능한 점 참고하기

4. tf-idf 과정을 통해 얻은 문장(단어) 중요도를 활용하여 ranking, 요약할 문장(단어) 수만큼 출력

공지 파일 결측치 41개 어떻게 처리할 것인가? --> 이미지의 손상으로 인해 데이터를 보완할 수 없기 때문에 삭제하기로 결정
- 만약 HUFS 공지 검색을 통해 보완할 수 있다면 최대한 보완할 것인가? 논의하기
- 보완 +로그 남기기 + 더이상 보완할 수 없을 경우 '직접 접속해서 확인해주세요.' 출력하기

2024.04.19
- 공지 파일 결측치 중 보완 가능한 23개 내용 보충 완료 후 재업로드
- 학사 인덱스 보충완료

In [ ]:
# 공지 csv의 결측치 파악
# df2[df2['content'] == '\n\n']

In [9]:
import pandas as pd
import os


In [13]:
#경로확인
print(os.getcwd())

/content


In [12]:
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)

### 마운트 및 파일 로드부터 시작하기 (경로 설정)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
gongji = pd.read_csv('./drive/MyDrive/csv모음/merge_gongji.csv')
gongji.head(5)

,index,date,title,content
0,1,2018-01-02,2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집,\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터...
1,2,2018-01-02,2018년 상반기 FLC 외국어교육과정 교육생 모집,\n2018년 상반기 FLC \n외국어교육과정 교육생 모집기초부터 고급까지! 해외연...
2,3,2018-01-02,2018년 상반기 10주 주말(토) 외국어교육과정 교육생 모집,\n2018년 상반기 \n10주 \n주말(토) 외국어교육과정 교육생 \n모집기초부터...
3,4,2018-01-03,대원빌딩 철거공사 입찰 관련 질의 응답,\n대원빌딩 철거공사 입찰 관련 질의 응답대원빌딩 철거공사 입찰과 관련한 질의사항에...
4,7,2018-01-04,2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고,\n2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고\n2018학...


In [20]:
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
haksa.head()

,index,date,title,content
0,1,2020-01-06,[공통] 2019-2학기 이중전공 변경 배정 확정 공고,\n\n\n\n\n\n[공통]이중전공변경 배정 확정 공고\n \n2019학년도 제2...
1,2,2020-01-13,[공통] 2020학년도 1학기 수강신청 공고,\n2020학년도 1학기 수강신청 공고 2020학년도 제1학기 수강신청에 대하여 아...
2,3,2020-01-13,[공통] 2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내,\n2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내\n \n2020학년도...
3,4,2020-01-13,[공통] 2020학년도 전기 일반대학원 학석사연계과정 모집 안내,\n2020학년도 전기 일반대학원 학석사연계과정 모집 안내\n\n2020학년도 전기...
4,5,2020-01-15,[글로벌]졸업 예비사정 결과 조회,\n[글로벌] 졸업 예비사정 결과 조회글로벌캠퍼스 졸업(2020년 2월) 예비사정 ...


In [21]:
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
hiring.head()

,index,date,title,content
0,1269,2024.03.28,국제학사 공로봉사장학생 모집 안내,\n국제학사 사감 모집 공고(여 대학원생) \n\n1. 모집 관련 일반사항\n가. ...
1,1268,2024.03.27,[사이버외대 영어학부] 전임조교 모집공지,\n[사이버외대 영어학부] 전임조교 모집공지\n\n사이버한국외국어대학교 영어학부에서...
2,1267,2024.03.27,중국연구소 전임연구원 모집,"\n중국연구소 전임연구원 모집\n\n안녕하세요,한국외국어대학교 중국연구소입니다.\n..."
3,1266,2024.03.26,한국외국어대학교 신입직원 채용공고,한국외국어대학교 신입직원 채용공고\n\n신입직원 채용공고\n2024.04.01(월)...
4,1265,2024.03.25,2024학년도 국제여름학기 Workstudy 장학생 모집 공고 (~4/8),\n2024학년도 국제여름학기 Workstudy 장학생 모집 공고 (~4/8)\n\...


In [22]:
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')
janghak.head()

,index,date,title,content
0,1,2020-01-02,[공통][교외] 2020-1학기 남정장학재단 장학생 모집 안내(~1/10),\n[공통][교외] 2020-1학기 남정장학재단 장학생 모집 안내(~1/10)아래와...
1,2,2020-01-02,[공통] [대출] 2020년 1학기 농촌출신대학생 학자금융자(무이자) 지원 안내,\n\n2020년 1학기 농촌출신대학생 학자금융자(무이자) 지원 안내\n\n \n사...
2,3,2020-01-07,[공통][교외] 2020년 상반기 시각장애대학생을 위한 [효명장학사업] 안내(~1/23),\n2020년 상반기 시각장애대학생을 위한 [효명장학사업] 안내 실로암시각장애인복지...
3,7,2020-01-08,[공통] [대출] 2020-1학기 한국장학재단 학자금대출 신청 안내,\n2020-1학기 한국장학재단 학자금대출 신청 안내\n \n1. 신청 및 실행 일...
4,4,2020-01-08,[공통] [대출] 2019년 하반기 천안시 대학생 학자금 대출이자 지원 안내,\n2019년 하반기 천안시 대학생 학자금 대출이자 지원 안내\n\n\n✿ 신청 기...


# re 이용해 특정 문자열 제거

In [23]:
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()

In [24]:
gongji_copy

,index,date,title,content
0,1,2018-01-02,2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집,\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터...
1,2,2018-01-02,2018년 상반기 FLC 외국어교육과정 교육생 모집,\n2018년 상반기 FLC \n외국어교육과정 교육생 모집기초부터 고급까지! 해외연...
2,3,2018-01-02,2018년 상반기 10주 주말(토) 외국어교육과정 교육생 모집,\n2018년 상반기 \n10주 \n주말(토) 외국어교육과정 교육생 \n모집기초부터...
3,4,2018-01-03,대원빌딩 철거공사 입찰 관련 질의 응답,\n대원빌딩 철거공사 입찰 관련 질의 응답대원빌딩 철거공사 입찰과 관련한 질의사항에...
4,7,2018-01-04,2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고,\n2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고\n2018학...
...,...,...,...,...
6708,4593,2024-03-25,FBS 외대교육방송국 78기 수습국원 추가모집,\nFBS 외대교육방송국 78기 수습국원 추가모집\n\n한국외국어대학교FBS외대교육...
6709,4594,2024-03-26,인도연구소 HK+사업단 제66회 명사초청 특강 안내,\n인도연구소 HK+사업단 제66회 명사초청 특강 안내\n\n□한국외국어대학교 인도...
6710,4595,2024-03-26,"EU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최","\nEU연구소 인문사회연구사업단, '제5회 바우하우스 콜로키움' 개최\n\n\nEU..."
6711,4596,2024-03-27,[서울] 보건실 - 심폐소생술 교육일정 안내 (2024.04.29(월) 10시 / ...,\n심폐소생술 교육일정 안내\n\n서울캠퍼스 보건실에서는 교내 구성원의 응급처치 능...


### 전처리과정
- 전번, 주소, 메일 등 정보 부터 지우는 중
- 가독성 때문에 띄어쓰기 및 특수기호는 문자열 지운 이후에 마지막에 지울 듯 함
- 현재 test만 지웠고 데이터 프레임에 적용하지 않았음

In [25]:
import re

In [27]:
gongji_copy.loc[0,'content']

'\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터 고급까지! 해외연수 그 이상의 효과\n실용 회화능력 집중 향상! 국내 최고의 외국어마스터과정\n\n한국외국어대학교 외국어연수평가원\n2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집\n\n[과정소개]\n‘FLC 외국어교육과정’은 대학(원)생, 취업준비생, 해외연수 및 유학준비생, 인사혁신처 추천 공무원 및 일반 공무원, 기관 및 기업체 임직원, 회사원, 일반인 등을 대상으로 말하기, 듣기, 읽기, 쓰기의 체계적이고 강도높은 교육을 통해 실제 의사소통능력을 집중적으로 향상시켜 해외연수 그 이상의 효과를 거둘 수 있는 국내 최고의 외국어마스터 과정으로 특히, 과정 이수 후 교육성과 및 만족도 면에서 높은 평가로 프로그램의 우수성을 널리 인정받고 있습니다.\n\n1. 교육일정\n[20주 주간/야간 정규과정] “말하기, 듣기, 쓰기, 읽기의 균형있는 통합교육”\n구 분주 간야 간접수기간2017. 12. 11(월) 부터~교육기간2018. 02. 05(월)~06. 22(금)교육시간09:30~16:201일 6시간/월~금/주5일18:40~21:301일 3시간/월~금/주5일총 교육시수총 600시간총 300시간반편성시험(Level Test)2018. 01. 20(토) 예정(장소:외국어연수평가원)교 육 비3,737,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)1,821,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)※단, 위 교육비는 일반적인 기준의 교육비이며, 인사혁신처 추천 공무원 및 별도 위탁교육생의 경우 교육비가 다를 수 있으므로 별도 문의\n\n2. 교육언어\n- 영어, 중국어, 일본어, 스페인어, 베트남어, 프랑스어, 독일어, 러시아어, 마인어 외\n- 과정별 특성에 따라 모집언어는 다를 수 있으며 모집공지시 자세한 교육언어 참조\n- 언어/레벨별 적정인원 모집시 개설되며 모집인원 미달시에는 개설되지

In [55]:
# 전화번호를 포함하는 패턴
tel_number = r'TEL\.\d{2}-?\d{3,4}-\d{4}'
#전번 양식 (번호만 있는 경우대비)
phone_number = r'\d{2}-\d{4}-\d{4}'
#팩스 양식
fax_number = r'FAX\.\d{2}-\d{3}-\d{4}'
#주소 양식
address = r'서울시 동대문구 이문로 107'
#이메일 양식
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
#링크 양식
link_http = r'http://\S+'
link_www = r'\bwww\.\S*'
link_hufs = r'hufs\.ac\.kr'

In [56]:
test = gongji_copy.loc[0,'content']
print(test)


2018년 상반기 
20주 
주간/야간 외국어교육과정 교육생 
모집기초부터 고급까지! 해외연수 그 이상의 효과
실용 회화능력 집중 향상! 국내 최고의 외국어마스터과정

한국외국어대학교 외국어연수평가원
2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집

[과정소개]
‘FLC 외국어교육과정’은 대학(원)생, 취업준비생, 해외연수 및 유학준비생, 인사혁신처 추천 공무원 및 일반 공무원, 기관 및 기업체 임직원, 회사원, 일반인 등을 대상으로 말하기, 듣기, 읽기, 쓰기의 체계적이고 강도높은 교육을 통해 실제 의사소통능력을 집중적으로 향상시켜 해외연수 그 이상의 효과를 거둘 수 있는 국내 최고의 외국어마스터 과정으로 특히, 과정 이수 후 교육성과 및 만족도 면에서 높은 평가로 프로그램의 우수성을 널리 인정받고 있습니다.

1. 교육일정
[20주 주간/야간 정규과정] “말하기, 듣기, 쓰기, 읽기의 균형있는 통합교육”
구 분주 간야 간접수기간2017. 12. 11(월) 부터~교육기간2018. 02. 05(월)~06. 22(금)교육시간09:30~16:201일 6시간/월~금/주5일18:40~21:301일 3시간/월~금/주5일총 교육시수총 600시간총 300시간반편성시험(Level Test)2018. 01. 20(토) 예정(장소:외국어연수평가원)교 육 비3,737,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)1,821,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)※단, 위 교육비는 일반적인 기준의 교육비이며, 인사혁신처 추천 공무원 및 별도 위탁교육생의 경우 교육비가 다를 수 있으므로 별도 문의

2. 교육언어
- 영어, 중국어, 일본어, 스페인어, 베트남어, 프랑스어, 독일어, 러시아어, 마인어 외
- 과정별 특성에 따라 모집언어는 다를 수 있으며 모집공지시 자세한 교육언어 참조
- 언어/레벨별 적정인원 모집시 개설되며 모집인원 미달시에는 개설되지 않음

3. 교육대상
- 대학(원)생

In [58]:
# 각 패턴에 대해 검열 수행
test = re.sub(tel_number, '', test)
test = re.sub(phone_number, '', test)
test = re.sub(fax_number, '', test)
test = re.sub(address,'',test)
test = re.sub(email_pattern,'',test)
test = re.sub(link_http,'',test)
test = re.sub(link_www,'',test)
test = re.sub(link_hufs,'',test)
print(test)


2018년 상반기 
20주 
주간/야간 외국어교육과정 교육생 
모집기초부터 고급까지! 해외연수 그 이상의 효과
실용 회화능력 집중 향상! 국내 최고의 외국어마스터과정

한국외국어대학교 외국어연수평가원
2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집

[과정소개]
‘FLC 외국어교육과정’은 대학(원)생, 취업준비생, 해외연수 및 유학준비생, 인사혁신처 추천 공무원 및 일반 공무원, 기관 및 기업체 임직원, 회사원, 일반인 등을 대상으로 말하기, 듣기, 읽기, 쓰기의 체계적이고 강도높은 교육을 통해 실제 의사소통능력을 집중적으로 향상시켜 해외연수 그 이상의 효과를 거둘 수 있는 국내 최고의 외국어마스터 과정으로 특히, 과정 이수 후 교육성과 및 만족도 면에서 높은 평가로 프로그램의 우수성을 널리 인정받고 있습니다.

1. 교육일정
[20주 주간/야간 정규과정] “말하기, 듣기, 쓰기, 읽기의 균형있는 통합교육”
구 분주 간야 간접수기간2017. 12. 11(월) 부터~교육기간2018. 02. 05(월)~06. 22(금)교육시간09:30~16:201일 6시간/월~금/주5일18:40~21:301일 3시간/월~금/주5일총 교육시수총 600시간총 300시간반편성시험(Level Test)2018. 01. 20(토) 예정(장소:외국어연수평가원)교 육 비3,737,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)1,821,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)※단, 위 교육비는 일반적인 기준의 교육비이며, 인사혁신처 추천 공무원 및 별도 위탁교육생의 경우 교육비가 다를 수 있으므로 별도 문의

2. 교육언어
- 영어, 중국어, 일본어, 스페인어, 베트남어, 프랑스어, 독일어, 러시아어, 마인어 외
- 과정별 특성에 따라 모집언어는 다를 수 있으며 모집공지시 자세한 교육언어 참조
- 언어/레벨별 적정인원 모집시 개설되며 모집인원 미달시에는 개설되지 않음

3. 교육대상
- 대학(원)생